In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

## Use Beautiful Soup to Get Match & Team Details (Upcoming Only)

In [2]:
url = "https://www.oddsportal.com/rugby-union/world/super-rugby/"
html = urlopen(url, timeout = 5)

In [3]:
soup = BeautifulSoup(html, 'lxml')

In [4]:
soup.find_all('tr',{'xeid':True})

[<tr class="odd" xeid="lAQoapK3"><td class="table-time datet t1580450700-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/blues-waikato-chiefs-lAQoapK3/">Blues - Chiefs</a></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr xeid="2DUkb4Z9"><td class="table-time datet t1580458500-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/act-brumbies-queensland-reds-2DUkb4Z9/">Brumbies - Reds</a></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="odds-nowrp"><span>-</span></td><td class="center info-value"></td></tr>,
 <tr class="odd" xeid="dzUgcOlG"><td class="table-time datet t1580490600-1-1-0-0"></td><td class="name table-participant" colspan="2"><a href="/rugby-union/world/super-rugby/sharks-blue-bulls-dzUgcOlG/">Sharks

In [5]:
games = soup.find_all(attrs={'class':'name table-participant'})

In [6]:
home_team = []
away_team = []

In [7]:
for game in games:
    teams = str(game).split('>')[2].split('<')[0].split(' - ')
    home_team.append(teams[0])
    away_team.append(teams[1])

## Use REST Service for Odds

In [8]:
import json
import requests
import objectpath

In [9]:
s = requests.Session()
request_headers = {
    'Referer': 'https://www.oddsportal.com/rugby-union/world/super-rugby/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}

In [10]:
s.get

<bound method Session.get of <requests.sessions.Session object at 0x000002948B975708>>

In [11]:
get_odds = s.get('https://fb.oddsportal.com/ajax-sport-country-tournament/8/OjguN6X6/X0/1/',
                  headers = request_headers)

In [12]:
text_odds = get_odds.text[72:-2]

In [13]:
json_odds = json.loads(text_odds)

In [14]:
json_tree = objectpath.Tree(json_odds['d'])

In [15]:
odds = tuple(json_tree.execute('$..avg'))
# odds all appear twice for some reason, so we take just half of them
odds = odds[:np.int(len(odds)/2)]

In [16]:
one_list = []
x_list = []
two_list = []

In [17]:
# take every 3 rows and put them into 1 row
for i in range(len(odds)):
    if i%3==0:
        one_list.append(odds[i])
    if i%3==1:
        x_list.append(odds[i])
    if i%3==2:
        two_list.append(odds[i])

In [18]:
# concatenate info into dataframe
df = pd.DataFrame({'Home Team':home_team,
                    'Away Team':away_team,
                    'Home Odds':one_list[:len(home_team)],
                    'Draw Odds':x_list[:len(home_team)],
                    'Away Odds':two_list[:len(home_team)]})

In [19]:
df.head()

,Home Team,Away Team,Home Odds,Draw Odds,Away Odds
0,Blues,Chiefs,2.17,21.35,1.71
1,Brumbies,Reds,1.38,23.23,3.06
2,Sharks,Bulls,1.52,20.93,2.58
3,Sunwolves,Rebels,6.60,34.84,1.10
4,Crusaders,Waratahs,1.08,36.10,7.45
